In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

### Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']


#determine the number of samples for validation (10% of the training data)
num_validation_samples = 0.1*mnist_info.splits['train'].num_examples 
#make sure the validation sample is an integer
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

#turn all inputs (0-255) into floats (0-1)
def scale(image,label):                 
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale) #scale train and validation data

test_data = mnist_test.map(scale) #scale test data

BUFFER_SIZE = 10000

#shuffle the data in BUFFER_SIZE increments
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

#take num_va_samples from the shuffled data as our validation set
validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
#turn the rest of the shuffled data into a training set
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

#hyperparameter that can be adjusted
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


### Model

#### Outlining the Model

In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 200
hidden_layer_size2 = 400

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),#flatten images into a vector
                            #tf.keras.layers.Dense(hidden_layer_size2, activation = 'relu'),#first hidden layer
                            tf.keras.layers.Dense(hidden_layer_size2, activation = 'relu'), #second hidden layer
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'), #third hidden layer
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'), #fourth hidden layer
                            tf.keras.layers.Dense(hidden_layer_size, activation = 'sigmoid'), #fifth hidden layer
                            tf.keras.layers.Dense(output_size, activation = 'softmax'), #outout layer                        
                            ])

#### Optimizer and Loss Functions

In [5]:
#custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer= 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

#### Training

In [6]:
NUM_EPOCHS = 15
early_stopping = tf.keras.callbacks.EarlyStopping(patience=1)

model.fit(train_data, 
          epochs=NUM_EPOCHS, 
          callbacks = [early_stopping],
          validation_data=(validation_inputs, validation_targets), 
          verbose=2)

Epoch 1/15
540/540 - 7s - loss: 0.2880 - accuracy: 0.9166 - val_loss: 0.1347 - val_accuracy: 0.9603
Epoch 2/15
540/540 - 5s - loss: 0.0946 - accuracy: 0.9712 - val_loss: 0.0777 - val_accuracy: 0.9760
Epoch 3/15
540/540 - 5s - loss: 0.0658 - accuracy: 0.9796 - val_loss: 0.0585 - val_accuracy: 0.9808
Epoch 4/15
540/540 - 5s - loss: 0.0474 - accuracy: 0.9849 - val_loss: 0.0512 - val_accuracy: 0.9847
Epoch 5/15
540/540 - 5s - loss: 0.0384 - accuracy: 0.9874 - val_loss: 0.0387 - val_accuracy: 0.9897
Epoch 6/15
540/540 - 5s - loss: 0.0294 - accuracy: 0.9910 - val_loss: 0.0368 - val_accuracy: 0.9898
Epoch 7/15
540/540 - 5s - loss: 0.0272 - accuracy: 0.9913 - val_loss: 0.0294 - val_accuracy: 0.9915
Epoch 8/15
540/540 - 5s - loss: 0.0267 - accuracy: 0.9916 - val_loss: 0.0296 - val_accuracy: 0.9915


#### Test the Model

In [7]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 1s 580ms/step - loss: 0.0796 - accuracy: 0.9799


In [8]:
print('Test loss: {0:.2f}; Test accuracy: {1:.2f}%'.format(test_loss,test_accuracy*100))

Test loss: 0.08; Test accuracy: 97.99%
